### Section 6: Apache Spark 1.6 - Data Analysis - Spark SQL or HiveQL using Spark Context

#### Different interfaces to run SQL - Hive, Spark SQL

* from command line
***
spark-sql --master yarn --conf spark.ui.port=12567
***
- in sparl sql prompt list databases
    * $show databases; 

- same queries can be run using hive prompt
    * $hive
- hive will be compiled to map reduce framework
- if its spark-sql will be compiled to spark framework
- to compile need to read metadata - called hive metastore

###  Create database and tables of text file format - orders and order_items

* Create database *retail_db*
* Create tables *orders* and *order_items*
* Load data into tables

In [ ]:
! hive

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 589e3a17-0278-4111-9f5d-a85dc9b0fd54

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.2.jar!/hive-log4j2.properties Async: true
Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
Hive Session ID = 0d5f858e-45db-4f03-8788-8316abde41e0
Hive-on-MR is deprecated in Hive 2 and may not be available in the futu

* launch hive in command prompt
***
hive
***
>set hive.metastore.warehouse.dir;
***
- above command shows the path of *.db creation
- hive.metastore.warehouse.dir=/user/hive/warehouse  --hdfs location
- Create database
***
>create database reatil_db_txt
***
>use retail_db_txt
***

In [2]:
! hdfs dfs -ls /user/hive/warehouse

2020-05-23 10:47:36,932 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
drwxr-xr-x   - pi supergroup          0 2020-05-23 10:44 /user/hive/warehouse/retail_db_txt.db


##### Create tables orders and order_items
* Orders
    - order_id int(11)
    - order_date datetime
    - order_custermer_id int(11)
    - order_status varchar(45)

* Order_Items
    - order_item_id int(11)
    - order_id int(11)
    - product_id int(11)
    - quantity tinyint(4)
    - subtotal float
    - product_price float


In [ ]:
hive> create table orders (
    > order_id int,
    > order_date string,
    > customer_id int,
    > status string
    > ) row format delimited fields terminated by ','
    > stored as textfile;
OK
Time taken: 1.759 seconds

In [ ]:
hive> show tables;
OK
orders
Time taken: 0.164 seconds, Fetched: 1 row(s)
hive> select * from orders;
OK
Time taken: 5.366 seconds
hive> select * from orders limit 10;
OK
Time taken: 0.534 seconds
hive>

In [ ]:
#load table
#from hdfs - without truncate load table
hive> load data inpath '/user/pi/retail_db/orders' into table orders;
#from hdfs with truncate load
hive> load data inpath '/user/pi/retail_db/orders' overwrite into table orders;
#from local - without truncate load table
hive> load data local inpath '/user/pi/retail_db/orders' into table orders;
#from local with truncate load
hive> load data local inpath '/user/pi/retail_db/orders' overwrite into table orders;
hive> dfs -ls /user/hive/warehouse/retail_db_txt.db/orders/;
Found 1 items
-rw-r--r--   2 pi supergroup    3068827 2020-05-11 12:05 /user/hive/warehouse/retail_db_txt.db/orders/part-00000
hive> select * from orders limit 10;
OK
1       2013-07-25 00:00:00.0   11599   CLOSED
2       2013-07-25 00:00:00.0   256     PENDING_PAYMENT
3       2013-07-25 00:00:00.0   12111   COMPLETE
4       2013-07-25 00:00:00.0   8827    CLOSED
5       2013-07-25 00:00:00.0   11318   COMPLETE
6       2013-07-25 00:00:00.0   7130    COMPLETE
7       2013-07-25 00:00:00.0   4530    COMPLETE
8       2013-07-25 00:00:00.0   2911    PROCESSING
9       2013-07-25 00:00:00.0   5657    PENDING_PAYMENT
10      2013-07-25 00:00:00.0   5648    PENDING_PAYMENT
Time taken: 0.485 seconds, Fetched: 10 row(s)

In [ ]:
#create order_items table
hive> create table order_items(
    >     order_item_id int,
    >     order_id int,
    >     product_id int,
    >     quantity int,
    >     subtotal float,
    >     product_price float)
    >     row format delimited fields terminated by ","
    >     stored as textfile;
OK
Time taken: 3.555 seconds
hive> load data inpath "/user/pi/retail_db/order_items" into table order_items;
Loading data to table default.order_items
OK
Time taken: 1.754 seconds
hive> select * From order_items limit 10;
OK
1       1       957     1       299.98  299.98
2       2       1073    1       199.99  199.99
3       2       502     5       250.0   50.0
4       2       403     1       129.99  129.99
5       4       897     2       49.98   24.99
6       4       365     5       299.95  59.99
7       4       502     3       150.0   50.0
8       4       1014    4       199.92  49.98
9       5       957     1       299.98  299.98
10      5       365     5       299.95  59.99
Time taken: 4.983 seconds, Fetched: 10 row(s)

###  Create database and tables of ORC file format - orders and order_items

In [6]:
#As our source data in text format ,we need to run insert command to convert data to ORC and store into tables in new Database
#Other tan text file format other formats like ORC,parquet,avro,RCFILE stores metadata along with data.
#Hence no need to mention delimiter explicitly while creating table.

In [ ]:
hive> describe formated order_items;
FAILED: SemanticException [Error 10001]: Table not found formated
hive> describe formatted order_items;
OK
# col_name              data_type               comment
order_item_id           int
order_id                int
product_id              int
quantity                int
subtotal                float
product_price           float

# Detailed Table Information
Database:               retail_db_txt
OwnerType:              USER
Owner:                  pi
CreateTime:             Sat May 23 11:58:49 BST 2020
LastAccessTime:         UNKNOWN
Retention:              0
Location:               hdfs://raspberrypi1:9000/user/hive/warehouse/retail_db_txt.db/order_items
Table Type:             MANAGED_TABLE
Table Parameters:
        COLUMN_STATS_ACCURATE   {\"BASIC_STATS\":\"true\",\"COLUMN_STATS\":{\"order_id\":\"true\",\"order_item_id\":\"true\",\"product_id\":\"true\",\"product_price\":\"true\",\uantity\":\"true\",\"subtotal\":\"true\"}}
        bucketing_version       2
        numFiles                0
        numRows                 0
        rawDataSize             0
        totalSize               0
        transient_lastDdlTime   1590232014

# Storage Information
SerDe Library:          org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe
InputFormat:            org.apache.hadoop.mapred.TextInputFormat
OutputFormat:           org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat
Compressed:             No
Num Buckets:            -1
Bucket Columns:         []
Sort Columns:           []
Storage Desc Params:
        field.delim             ,
        serialization.format    ,
Time taken: 0.292 seconds, Fetched: 37 row(s)

In [ ]:
hive> create database retail_db_orc;
hive> use retail_db_orc;
OK
Time taken: 0.121 seconds
hive> create table orders (
    > order_id int,
    > order_date string,
    > customer_id int,
    > status string
    > ) stored as orc;
OK
Time taken: 0.253 seconds
hive> create table order_items(
    >     order_item_id int,
    >     order_id int,
    >     product_id int,
    >     quantity int,
    >     subtotal float,
    >     product_price float) stored as orc;
OK
Time taken: 0.289 seconds
#since the input file in text format we can user insert to insert to table in orc format seleting from text db "retail_db_txt"
hive> insert into orders select * from retail_db_txt.orders;
hive> select * from orders limit 5;
OK
1       2013-07-25 00:00:00.0   11599   CLOSED
2       2013-07-25 00:00:00.0   256     PENDING_PAYMENT
3       2013-07-25 00:00:00.0   12111   COMPLETE
4       2013-07-25 00:00:00.0   8827    CLOSED
5       2013-07-25 00:00:00.0   11318   COMPLETE
Time taken: 0.513 seconds, Fetched: 5 row(s)
hive> select * from order_items limit 10;
OK
1       1       957     1       299.98  299.98
2       2       1073    1       199.99  199.99
3       2       502     5       250.0   50.0
4       2       403     1       129.99  129.99
5       4       897     2       49.98   24.99
6       4       365     5       299.95  59.99
7       4       502     3       150.0   50.0
8       4       1014    4       199.92  49.98
9       5       957     1       299.98  299.98
10      5       365     5       299.95  59.99
Time taken: 0.461 seconds, Fetched: 10 row(s)

### Running SQL/Hive Commands using pyspark

In [2]:
#Hive queries can be executed from pyspark using sqlContext.sql()
#result willbe form dataframe
#to display show()
#or convert to collection using collect()

In [3]:
sqlContext

In [7]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [ ]:
>>> sqlContext.sql("show databases").show()
+-------------+
| databaseName|
+-------------+
|      default|
|retail_db_txt|
+-------------+
>>> sqlContext.sql("use retail_db_txt")
DataFrame[]
>>> sqlContext.sql("create table orders ( \
... order_id int,\
... order_date string,\
... customer_id int,\
... status string\
... ) row format delimited fields terminated by ','\
...     stored as textfile")

2020-05-23 13:47:35,731 WARN metastore.HiveMetaStore: Location: file:/home/pi/spark-warehouse/retail_db_txt.db/orders specified for non-external table:orders
DataFrame[]
>>> sqlContext.sql("load data inpath '/user/pi/retail_db/orders' into table orders")
DataFrame[]
>>> sqlContext.sql("select * from orders limit 10").show()
+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|         status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
+--------+--------------------+-----------+---------------+
>>> for rec in sqlContext.sql("select * from orders limit 10").collect():
...     print(rec)
Row(order_id=1, order_date='2013-07-25 00:00:00.0', customer_id=11599, status='CLOSED')
Row(order_id=2, order_date='2013-07-25 00:00:00.0', customer_id=256, status='PENDING_PAYMENT')
Row(order_id=3, order_date='2013-07-25 00:00:00.0', customer_id=12111, status='COMPLETE')
Row(order_id=4, order_date='2013-07-25 00:00:00.0', customer_id=8827, status='CLOSED')
Row(order_id=5, order_date='2013-07-25 00:00:00.0', customer_id=11318, status='COMPLETE')
Row(order_id=6, order_date='2013-07-25 00:00:00.0', customer_id=7130, status='COMPLETE')
Row(order_id=7, order_date='2013-07-25 00:00:00.0', customer_id=4530, status='COMPLETE')
Row(order_id=8, order_date='2013-07-25 00:00:00.0', customer_id=2911, status='PROCESSING')
Row(order_id=9, order_date='2013-07-25 00:00:00.0', customer_id=5657, status='PENDING_PAYMENT')
Row(order_id=10, order_date='2013-07-25 00:00:00.0', customer_id=5648, status='PENDING_PAYMENT')


* Other concepts
    - Filtering (horizontal and vertical)
    - Functions
    - Row level Transformations
    - Joins
    - Aggregation
    - Sorting
    - Set Operations
    - Analystics Function
    - Windowing Functions

### Functions - Getting Started

- hive> show functions;
***
OK| !| !=| $sum0| %| &| *| +| -| /| <| <=| <=>| <>| =| ==| >| >=| ^| abs| acos| add_months| aes_decrypt| aes_encrypt| and| array| array_contains| ascii| asin| assert_true| assert_true_oom| atan| avg| base64| between| bin| bloom_filter| bround| cardinality_violation| case| cbrt| ceil| ceiling| char_length| character_length| chr| coalesce| collect_list| collect_set| compute_stats| concat| concat_ws| context_ngrams| conv| corr| cos| count| covar_pop| covar_samp| crc32| create_union| cume_dist| current_authorizer| current_database| current_date| current_groups| current_timestamp| current_user| date_add| date_format| date_sub| datediff| day| dayofmonth| dayofweek| decode| degrees| dense_rank| div| e| elt| encode| enforce_constraint| exp| explode| extract_union| factorial| field| find_in_set| first_value| floor| floor_day| floor_hour| floor_minute| floor_month| floor_quarter| floor_second| floor_week| floor_year| format_number| from_unixtime| from_utc_timestamp| get_json_object| get_splits| greatest| grouping| hash| hex| histogram_numeric| hour| if| in| in_bloom_filter| in_file| index| initcap| inline| instr| internal_interval| isfalse| isnotfalse| isnotnull| isnottrue| isnull| istrue| java_method| json_tuple| lag| last_day| last_value| lcase| lead| least| length| levenshtein| like| likeall| likeany| ln| locate| log| log10| log2| logged_in_user| lower| lpad| ltrim| map| map_keys| map_values| mask| mask_first_n| mask_hash| mask_last_n| mask_show_first_n| mask_show_last_n| matchpath| max| md5| min| minute| mod| month| months_between| murmur_hash| named_struct| negative| next_day| ngrams| noop| noopstreaming| noopwithmap| noopwithmapstreaming| not| ntile| nullif| nvl| octet_length| or| parse_url| parse_url_tuple| percent_rank| percentile| percentile_approx| pi| pmod| posexplode| positive| pow| power| printf| quarter| radians| rand| rank| reflect| reflect2| regexp| regexp_extract| regexp_replace| regr_avgx| regr_avgy| regr_count| regr_intercept| regr_r2| regr_slope| regr_sxx| regr_sxy| regr_syy| repeat| replace| replicate_rows| restrict_information_schema| reverse| rlike| round| row_number| rpad| rtrim| second| sentences| sha| sha1| sha2| shiftleft| shiftright| shiftrightunsigned| sign| sin| size| sort_array| sort_array_by| soundex| space| split| sq_count_check| sqrt| stack| std| stddev| stddev_pop| stddev_samp| str_to_map| struct| substr| substring| substring_index| sum| tan| to_date| to_epoch_milli| to_unix_timestamp| to_utc_timestamp| translate| trim| trunc| ucase| udftoboolean| udftobyte| udftodouble| udftofloat| udftointeger| udftolong| udftoshort| udftostring| unbase64| unhex| unix_timestamp| upper| uuid| var_pop| var_samp| variance| version| weekofyear| when| width_bucket| windowingtablefunction| xpath| xpath_boolean| xpath_double| xpath_float| xpath_int| xpath_long| xpath_number| xpath_short| xpath_string| year| || ~| 
***

In [ ]:
#to get syntax
hive> describe function length;
OK
length(str | binary) - Returns the length of str or number of bytes in binary data
Time taken: 0.109 seconds, Fetched: 1 row(s)

In [14]:
sqlContext.sql("drop database retail_db_txt")
sqlContext.sql("create database retail_db_txt")
sqlContext.sql("use retail_db_txt")
sqlContext.sql("create table orders ( \
  order_id int,\
  order_date string,\
  customer_id int,\
  status string\
  ) row format delimited fields terminated by ','\
      stored as textfile")

DataFrame[]

In [15]:
sqlContext.sql("load data local inpath '/home/pi/shared/retail_db/orders' into table orders")

DataFrame[]

In [16]:
sqlContext.sql("select * from orders limit 10").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|         status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
+--------+--------------------+-----------+---------------+



In [13]:
sqlContext.sql("select status,count(*) from orders group by status").show()

+---------------+--------+
|         status|count(1)|
+---------------+--------+
|PENDING_PAYMENT|   30060|
|       COMPLETE|   45798|
|        ON_HOLD|    7596|
| PAYMENT_REVIEW|    1458|
|     PROCESSING|   16550|
|         CLOSED|   15112|
|SUSPECTED_FRAUD|    3116|
|        PENDING|   15220|
|       CANCELED|    2856|
+---------------+--------+



In [82]:
sqlContext.sql("create table order_items(\
        order_item_id int,\
        order_id int,\
        product_id int,\
        quantity int,\
        subtotal float,\
        product_price float) row format delimited fields terminated by ','\
        stored as textfile");

In [92]:
sqlContext.sql("load data local inpath '/home/pi/shared/retail_db/order_items' into table order_items").show()

++
||
++
++



In [93]:
sqlContext.sql("select * from order_items").show()

+-------------+--------+----------+--------+--------+-------------+
|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+-------------+--------+----------+--------+--------+-------------+
|            1|       1|       957|       1|  299.98|       299.98|
|            2|       2|      1073|       1|  199.99|       199.99|
|            3|       2|       502|       5|   250.0|         50.0|
|            4|       2|       403|       1|  129.99|       129.99|
|            5|       4|       897|       2|   49.98|        24.99|
|            6|       4|       365|       5|  299.95|        59.99|
|            7|       4|       502|       3|   150.0|         50.0|
|            8|       4|      1014|       4|  199.92|        49.98|
|            9|       5|       957|       1|  299.98|       299.98|
|           10|       5|       365|       5|  299.95|        59.99|
|           11|       5|      1014|       2|   99.96|        49.98|
|           12|       5|       957|       1|  29

### Functions - String Manipulation

In [36]:
sqlContext.sql("create table customers ( \
customer_id int,\
customer_fname varchar(45),\
customer_lname varchar(45),\
customer_email varchar(45),\
customer_password varchar(45),\
customer_street varchar(255),\
customer_city varchar(45),\
customer_state varchar(45),\
customer_zipcode varchar(45)) row format delimited fields terminated by ','\
stored as textfile")

DataFrame[]

In [37]:
sqlContext.sql('load data local inpath "/home/pi/shared/retail_db/customers" into table customers')

DataFrame[]

In [39]:
sqlContext.sql("select * from customers limit 5").show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

##### Important string functions
- substr or substring
- instr
- like
- rlike
- length
- lcase or lower
- ucase or upper
- trim,ltrim,rtrim
- lpad/rpad
- cast
- split/index
- initcap

In [30]:
sqlContext.sql("select substr('hello how are you',7,3) substr").show()

+------+
|substr|
+------+
|   how|
+------+



In [32]:
sqlContext.sql("select instr('hello how are you','are') instr").show()

+-----+
|instr|
+-----+
|   11|
+-----+



In [34]:
sqlContext.sql("select 'hello how are you' like '%how%'").show()

+----------------------------+
|hello how are you LIKE %how%|
+----------------------------+
|                        true|
+----------------------------+



In [44]:
sqlContext.sql("select '0501986734r' rlike '[^0-9]{2}'").show()

+---------------------------+
|0501986734r RLIKE [^0-9]{2}|
+---------------------------+
|                      false|
+---------------------------+



In [46]:
sqlContext.sql("select cast('0501986734' as int)").show()

+-----------------------+
|CAST(0501986734 AS INT)|
+-----------------------+
|              501986734|
+-----------------------+



In [49]:
sqlContext.sql("select split('hello how are you',' ')").collect()

[Row(split(hello how are you,  )=['hello', 'how', 'are', 'you'])]

In [59]:
sqlContext.sql("select split('hello how are you',' ') ").collect()

[Row(split(hello how are you,  )=['hello', 'how', 'are', 'you'])]

In [60]:
sqlContext.sql("describe function index").show()

+--------------------+
|       function_desc|
+--------------------+
|Function: index n...|
+--------------------+



In [62]:
sqlContext.sql("select index(split('1,2,3,4',','),2)").show()

AnalysisException: "Undefined function: 'index'. This function is neither a registered temporary function nor a permanent function registered in the database 'retail_db_txt'.; line 1 pos 7"

### Functions - Date Manipulation
***
	- current_date
	- current_timestamp
	- date_add
	- date_sub
	- date_format
	- datediff
	- day
	- dayofmonth
	- to_date
	- to_unix_timestamp
	- to_utc_timestamp
	- from_unixtime
	- from_utc_timestamp
	- minute
	- month
	- months_between
	- next_day
***

In [65]:
sqlContext.sql("select current_date").show()

+--------------+
|current_date()|
+--------------+
|    2020-05-23|
+--------------+



In [67]:
sqlContext.sql("select current_timestamp").show()

+--------------------+
| current_timestamp()|
+--------------------+
|2020-05-23 19:47:...|
+--------------------+



In [69]:
sqlContext.sql("select date_format(current_timestamp,'y')").show()

+-----------------------------------+
|date_format(current_timestamp(), y)|
+-----------------------------------+
|                               2020|
+-----------------------------------+



In [71]:
sqlContext.sql("select date_add(current_timestamp,10)").show()

+-----------------------------------------------+
|date_add(CAST(current_timestamp() AS DATE), 10)|
+-----------------------------------------------+
|                                     2020-06-02|
+-----------------------------------------------+



In [74]:
sqlContext.sql("describe function date_sub").collect()

[Row(function_desc='Function: date_sub'),
 Row(function_desc='Class: org.apache.spark.sql.catalyst.expressions.DateSub'),
 Row(function_desc='Usage: date_sub(start_date, num_days) - Returns the date that is `num_days` before `start_date`.')]

In [76]:
sqlContext.sql("describe function next_day").collect()

[Row(function_desc='Function: next_day'),
 Row(function_desc='Class: org.apache.spark.sql.catalyst.expressions.NextDay'),
 Row(function_desc='Usage: next_day(start_date, day_of_week) - Returns the first date which is later than `start_date` and named as indicated.')]

In [83]:
sqlContext.sql("select day(current_date)").show()

+--------------------------+
|dayofmonth(current_date())|
+--------------------------+
|                        23|
+--------------------------+



In [86]:
sqlContext.sql("select minute(current_timestamp)").show()

+---------------------------+
|minute(current_timestamp())|
+---------------------------+
|                         53|
+---------------------------+



In [88]:
sqlContext.sql("select month(current_timestamp)").show()

+----------------------------------------+
|month(CAST(current_timestamp() AS DATE))|
+----------------------------------------+
|                                       5|
+----------------------------------------+



In [90]:
sqlContext.sql("select to_unix_timestamp(current_timestamp)").show()

+-----------------------------------------------------------+
|to_unix_timestamp(current_timestamp(), yyyy-MM-dd HH:mm:ss)|
+-----------------------------------------------------------+
|                                                 1590260058|
+-----------------------------------------------------------+



In [92]:
sqlContext.sql("select from_unixtime(1590260058)").show()

+--------------------------------------------------------------+
|from_unixtime(CAST(1590260058 AS BIGINT), yyyy-MM-dd HH:mm:ss)|
+--------------------------------------------------------------+
|                                           2020-05-23 19:54:18|
+--------------------------------------------------------------+



###  Functions - Aggregate Functions in brief
***
    - count
    - avg
    - min
    - max
    - sum
***

In [108]:
sqlContext.sql("select count(*) total_records,count(distinct status) st from orders").show()

+-------------+---+
|total_records| st|
+-------------+---+
|       206649|  9|
+-------------+---+



### Functions - case and nvl
***
    - case
    - nvl
***

In [17]:
sqlContext.sql("select * from orders limit 10").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|         status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
+--------+--------------------+-----------+---------------+



In [122]:
sqlContext.sql("select case status when 'CLOSED' then 'COMPLETED'\
                                   when 'COMPLETE' then 'COMPLETED'\
                                   else 'PENDING' END status,count(*) from orders group by case status when 'CLOSED' then 'COMPLETED'\
                                   when 'COMPLETE' then 'COMPLETED'\
                                   else 'PENDING' END").show()

+---------+--------+
|   status|count(1)|
+---------+--------+
|COMPLETED|   91365|
|  PENDING|  115284|
+---------+--------+



In [123]:
sqlContext.sql("describe function case").collect()

[Row(function_desc='Function: case'),
 Row(function_desc='Usage: CASE expr1 WHEN expr2 THEN expr3 [WHEN expr4 THEN expr5]* [ELSE expr6] END - When `expr1` = `expr2`, returns `expr3`; when `expr1` = `expr4`, return `expr5`; else return `expr6`.')]

In [125]:
sqlContext.sql("select case  when status in ('CLOSED','COMPLETE') then 'COMPLETED'\
                                   else 'PENDING' END status,count(*) from orders group by case  when status in ('CLOSED','COMPLETE') then 'COMPLETED'\
                                   else 'PENDING' END").show()

+---------+--------+
|   status|count(1)|
+---------+--------+
|COMPLETED|   91365|
|  PENDING|  115284|
+---------+--------+



In [131]:
sqlContext.sql("select nvl(null,'Missing')").show()

+--------------------+
|nvl(NULL, 'Missing')|
+--------------------+
|             Missing|
+--------------------+



### Row level transformations

In [29]:
sqlContext.sql("show databases").show()

+-------------+
| databaseName|
+-------------+
|      default|
|retail_db_txt|
+-------------+



In [33]:
sqlContext.sql("select cast(concat(substr(order_date,1,4),substr(order_date,6,2)) as int) mon From orders limit 10").show()

+------+
|   mon|
+------+
|201307|
|201307|
|201307|
|201307|
|201307|
|201307|
|201307|
|201307|
|201307|
|201307|
+------+



In [35]:
sqlContext.sql("select cast(date_format(order_date,'yyyymm') as int) mon From orders limit 10").show()

+------+
|   mon|
+------+
|201300|
|201300|
|201300|
|201300|
|201300|
|201300|
|201300|
|201300|
|201300|
|201300|
+------+



### Joining data between multiple tables

In [42]:
sqlContext.sql("select * from customers limit 5").show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [46]:
sqlContext.sql("select * from orders limit 5").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|         status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
+--------+--------------------+-----------+---------------+



* types of joins are
    - join
    - left outer join
    - right outer join
    - full outer join

In [56]:
sqlContext.sql("select c.*,o.* from customers c,orders o where c.customer_id=o.customer_id limit 10").show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------+---------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|order_id|          order_date|customer_id|         status|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------+---------------+
|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|        256|         David|     Rodriguez|     XXXXXXXXX|        XXXXXXXXX|7605 Tawny Horse ...|      Chicago|            IL|           60625|       2|2013-07-25 00:00:...|        256|PENDING

In [58]:
sqlContext.sql("select c.*,o.* from customers c join orders o on c.customer_id=o.customer_id limit 10").show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------+---------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|order_id|          order_date|customer_id|         status|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+--------------------+-----------+---------------+
|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|        256|         David|     Rodriguez|     XXXXXXXXX|        XXXXXXXXX|7605 Tawny Horse ...|      Chicago|            IL|           60625|       2|2013-07-25 00:00:...|        256|PENDING

In [79]:
sqlContext.sql("select count(*) from customers c left outer join orders o on c.customer_id=o.customer_id").show()

+--------+
|count(1)|
+--------+
|   68913|
+--------+



In [81]:
#customer who didnt place orders yet
sqlContext.sql("select count(*) from customers c left outer join orders o on c.customer_id=o.customer_id where o.customer_id is null").show()

+--------+
|count(1)|
+--------+
|      30|
+--------+



### Group by and aggregations

- count()

In [94]:
#daily revenue for complete and closed orders
sqlContext.sql("select * from order_items").show()

+-------------+--------+----------+--------+--------+-------------+
|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+-------------+--------+----------+--------+--------+-------------+
|            1|       1|       957|       1|  299.98|       299.98|
|            2|       2|      1073|       1|  199.99|       199.99|
|            3|       2|       502|       5|   250.0|         50.0|
|            4|       2|       403|       1|  129.99|       129.99|
|            5|       4|       897|       2|   49.98|        24.99|
|            6|       4|       365|       5|  299.95|        59.99|
|            7|       4|       502|       3|   150.0|         50.0|
|            8|       4|      1014|       4|  199.92|        49.98|
|            9|       5|       957|       1|  299.98|       299.98|
|           10|       5|       365|       5|  299.95|        59.99|
|           11|       5|      1014|       2|   99.96|        49.98|
|           12|       5|       957|       1|  29

In [96]:
sqlContext.sql("select * from orders").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|         status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [138]:
sqlContext.sql("select to_date(order_date) order_date,round(sum(subtotal),2) revenue \
from orders o join order_items oi \
on o.order_id=oi.order_id \
where o.status in ('CLOSED','COMPLETE')\
group by to_date(order_date)").show()

+----------+--------+
|order_date| revenue|
+----------+--------+
|2013-09-09|49372.42|
|2013-09-19|51600.34|
|2014-06-03| 29006.2|
|2013-09-12|42801.83|
|2014-01-24|33855.51|
|2014-02-16|55914.86|
|2014-06-11|37022.75|
|2013-11-18|44898.19|
|2014-02-18|45412.09|
|2013-08-14|42043.45|
|2013-10-05|43319.77|
|2014-07-04|25467.71|
|2014-07-06|16451.76|
|2013-09-18|49899.08|
|2013-09-20|29575.36|
|2013-09-25|61042.05|
|2014-06-13|52380.18|
|2013-11-23|55605.42|
|2013-09-14|60928.16|
|2014-02-24|38936.61|
+----------+--------+
only showing top 20 rows



### Sorting the data

In [155]:
sqlContext.sql("select to_date(order_date) order_date,round(sum(subtotal),2) revenue \
from orders o join order_items oi \
on o.order_id=oi.order_id \
where o.status in ('CLOSED','COMPLETE')\
group by to_date(order_date)\
order by order_date,revenue desc").show()

+----------+--------+
|order_date| revenue|
+----------+--------+
|2013-07-25|31547.23|
|2013-07-26|54713.23|
|2013-07-27|48411.48|
|2013-07-28|35672.03|
|2013-07-29| 54579.7|
|2013-07-30|49329.29|
|2013-07-31|59212.49|
|2013-08-01|49160.08|
|2013-08-02|50688.58|
|2013-08-03|43416.74|
|2013-08-04|35093.01|
|2013-08-05|34025.27|
|2013-08-06|57843.89|
|2013-08-07|45525.59|
|2013-08-08|33549.47|
|2013-08-09|29225.16|
|2013-08-10|46435.04|
|2013-08-11| 31155.5|
|2013-08-12|59014.74|
|2013-08-13|17956.88|
+----------+--------+
only showing top 20 rows



In [158]:
#distribute by  - sort by 
#in above query if order by order_date is not taht important ,but within in order_date need to sort by revenue
sqlContext.sql("select to_date(order_date) order_date,round(sum(subtotal),2) revenue \
from orders o join order_items oi \
on o.order_id=oi.order_id \
where o.status in ('CLOSED','COMPLETE')\
group by to_date(order_date)\
distribute by order_date sort by order_date,revenue desc").show()

+----------+--------+
|order_date| revenue|
+----------+--------+
|2013-09-09|49372.42|
|2013-09-19|51600.34|
|2014-06-03| 29006.2|
|2013-09-12|42801.83|
|2014-01-24|33855.51|
|2014-02-16|55914.86|
|2014-06-11|37022.75|
|2013-11-18|44898.19|
|2014-02-18|45412.09|
|2013-08-14|42043.45|
|2013-10-05|43319.77|
|2014-07-04|25467.71|
|2014-07-06|16451.76|
|2013-09-18|49899.08|
|2013-09-20|29575.36|
|2013-09-25|61042.05|
|2014-06-13|52380.18|
|2013-11-23|55605.42|
|2013-09-14|60928.16|
|2014-02-24|38936.61|
+----------+--------+
only showing top 20 rows



### Set operations - union and union all

* union
* union all
    - both same logic in oracle

### Analytics functions - aggregations

In [199]:
sqlContext.sql("select * from orders o join order_items oi on o.order_id=oi.order_id where oi.order_id=2").show()

+--------+--------------------+-----------+---------------+-------------+--------+----------+--------+--------+-------------+
|order_id|          order_date|customer_id|         status|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+--------+--------------------+-----------+---------------+-------------+--------+----------+--------+--------+-------------+
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|            2|       2|      1073|       1|  199.99|       199.99|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|            3|       2|       502|       5|   250.0|         50.0|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|            4|       2|       403|       1|  129.99|       129.99|
+--------+--------------------+-----------+---------------+-------------+--------+----------+--------+--------+-------------+



In [258]:
sqlContext.sql("select o.order_id,to_date(o.order_date) order_date,oi.subtotal, \
round((subtotal/(sum(subtotal) over (partition by o.order_id)))*100,2) percentage_total \
from orders o join order_items oi on o.order_id=oi.order_id where oi.order_id=2").show()

+--------+----------+--------+----------------+
|order_id|order_date|subtotal|percentage_total|
+--------+----------+--------+----------------+
|       2|2013-07-25|  199.99|           34.48|
|       2|2013-07-25|   250.0|            43.1|
|       2|2013-07-25|  129.99|           22.41|
+--------+----------+--------+----------------+



In [259]:
#in hive subquery always to be aliased
#get those order have percentate total as 100
sqlContext.sql("select * from (select o.order_id,to_date(o.order_date) order_date,oi.subtotal, \
round((subtotal/(sum(subtotal) over (partition by o.order_id)))*100,2) percentage_total \
from orders o join order_items oi on o.order_id=oi.order_id) q where percentage_total=100").show()

+--------+----------+--------+----------------+
|order_id|order_date|subtotal|percentage_total|
+--------+----------+--------+----------------+
|    1580|2013-08-02|  299.95|           100.0|
|    2366|2013-08-06|  299.97|           100.0|
|    3749|2013-08-15|  143.97|           100.0|
|    3794|2013-08-16|  299.95|           100.0|
|    4101|2013-08-18|  129.99|           100.0|
|    4519|2013-08-21|   79.98|           100.0|
|    4818|2013-08-23|  399.98|           100.0|
|    5518|2013-08-27|  199.95|           100.0|
|    6397|2013-09-03|  399.98|           100.0|
|    6620|2013-09-05|  399.98|           100.0|
|    7554|2013-09-09|  199.99|           100.0|
|    7880|2013-09-11|   99.96|           100.0|
|    9900|2013-09-25|   39.99|           100.0|
|   10817|2013-09-29|   100.0|           100.0|
|   11141|2013-10-02|  399.98|           100.0|
|   12940|2013-10-12|  149.94|           100.0|
|   14570|2013-10-25|   39.99|           100.0|
|   15619|2013-11-01|   150.0|          

### Analytics functions - ranking

In [290]:
sqlContext.sql("select * from (select o.order_id,to_date(o.order_date) order_date,oi.subtotal, \
round((subtotal/(sum(subtotal) over (partition by o.order_id)))*100,2) percentage_total, \
rank() over (partition by o.order_id order by oi.subtotal desc) rnk_revenue, \
dense_rank() over (partition by o.order_id order by oi.subtotal desc) dense_rnk_revenue, \
percent_rank() over (partition by o.order_id order by oi.subtotal desc) pct_rnk_revenue, \
row_number() over (partition by o.order_id order by oi.subtotal desc) rn_orderid_revenue, \
row_number() over (order by o.order_id) rn_revenue \
from orders o join order_items oi on o.order_id=oi.order_id) q where order_id=5").show(100)

+--------+----------+--------+----------------+-----------+-----------------+---------------+------------------+----------+
|order_id|order_date|subtotal|percentage_total|rnk_revenue|dense_rnk_revenue|pct_rnk_revenue|rn_orderid_revenue|rn_revenue|
+--------+----------+--------+----------------+-----------+-----------------+---------------+------------------+----------+
|       5|2013-07-25|  299.98|           26.55|          1|                1|            0.0|                 1|         9|
|       5|2013-07-25|  299.98|           26.55|          1|                1|            0.0|                 2|        10|
|       5|2013-07-25|  299.95|           26.55|          3|                2|            0.5|                 3|        11|
|       5|2013-07-25|  129.99|            11.5|          4|                3|           0.75|                 4|        12|
|       5|2013-07-25|   99.96|            8.85|          5|                4|            1.0|                 5|        13|
+-------

### Windowing functions

- LEAD
- LAG
- FIRST_VALUE
- LAST_VALUE

In [341]:
sqlContext.sql("select * from (select o.order_id,to_date(o.order_date) ord_dt,oi.subtotal, \
round((subtotal/(sum(subtotal) over (partition by o.order_id)))*100,2) per_total, \
rank() over (partition by o.order_id order by oi.subtotal desc) rnk_rev, \
dense_rank() over (partition by o.order_id order by oi.subtotal desc) dns_rnk_rev, \
percent_rank() over (partition by o.order_id order by oi.subtotal desc) pct_rnk_rev, \
row_number() over (partition by o.order_id order by oi.subtotal desc) rn_ordid_rev, \
row_number() over (order by o.order_id) rn_rev, \
lead(oi.subtotal) over (partition by o.order_id order by oi.subtotal) lead_subtot, \
lag(oi.subtotal) over (partition by o.order_id order by oi.subtotal) lag_subtot, \
first_value(oi.subtotal) over (partition by o.order_id order by oi.subtotal) f_val, \
last_value(oi.subtotal) over (partition by o.order_id order by oi.subtotal) l_val \
from orders o join order_items oi on o.order_id=oi.order_id) q where order_id=5 \
order by subtotal desc").show(100)

+--------+----------+--------+---------+-------+-----------+-----------+------------+------+-----------+----------+-----+------+
|order_id|    ord_dt|subtotal|per_total|rnk_rev|dns_rnk_rev|pct_rnk_rev|rn_ordid_rev|rn_rev|lead_subtot|lag_subtot|f_val| l_val|
+--------+----------+--------+---------+-------+-----------+-----------+------------+------+-----------+----------+-----+------+
|       5|2013-07-25|  299.98|    26.55|      1|          1|        0.0|           1|     9|     299.98|    299.95|99.96|299.98|
|       5|2013-07-25|  299.98|    26.55|      1|          1|        0.0|           2|    12|       null|    299.98|99.96|299.98|
|       5|2013-07-25|  299.95|    26.55|      3|          2|        0.5|           3|    10|     299.98|    129.99|99.96|299.95|
|       5|2013-07-25|  129.99|     11.5|      4|          3|       0.75|           4|    13|     299.95|     99.96|99.96|129.99|
|       5|2013-07-25|   99.96|     8.85|      5|          4|        1.0|           5|    11|     

### Creating Data Frames and register as temp tables

* Get daily revenue by product considering completed and closed orders.
	- PRODUCTS have to be read from local file system.DataFrame need to be created
	- Join Orders,ORDER_ITEMS
	- Filter on ORDER_STATUS
* Data need to be sorted by ascending order by date and thendescending order by revenue computed for each product for each day
	- Sort data by order_date in ascending order and then daily revenue per product in descending order

In [3]:
sqlContext.sql("show databases").show()

+-------------+
| databaseName|
+-------------+
|      default|
|retail_db_txt|
+-------------+



In [4]:
sqlContext.sql("use retail_db_txt")

DataFrame[]

In [6]:
#to print Dataframe model
sqlContext.sql("select * from orders").printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- status: string (nullable = true)



#### Read data from HDFS to RDD and then convert to Dataframe and then to temptable

In [8]:
#read data from HDFS
orders=sc.textFile("/user/pi/retail_db/orders")
orders.take(10)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '8,2013-07-25 00:00:00.0,2911,PROCESSING',
 '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT',
 '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

In [11]:
#convert to RDD
from pyspark.sql import Row
ordersDF = orders.map(lambda o:Row(order_id=int(o.split(",")[0]),order_date=o.split(',')[1].split(" ")[0], \
                                   customer_id=int(o.split(",")[2]),order_status=o.split(",")[3])).toDF()

In [12]:
ordersDF.show()

+-----------+----------+--------+---------------+
|customer_id|order_date|order_id|   order_status|
+-----------+----------+--------+---------------+
|      11599|2013-07-25|       1|         CLOSED|
|        256|2013-07-25|       2|PENDING_PAYMENT|
|      12111|2013-07-25|       3|       COMPLETE|
|       8827|2013-07-25|       4|         CLOSED|
|      11318|2013-07-25|       5|       COMPLETE|
|       7130|2013-07-25|       6|       COMPLETE|
|       4530|2013-07-25|       7|       COMPLETE|
|       2911|2013-07-25|       8|     PROCESSING|
|       5657|2013-07-25|       9|PENDING_PAYMENT|
|       5648|2013-07-25|      10|PENDING_PAYMENT|
|        918|2013-07-25|      11| PAYMENT_REVIEW|
|       1837|2013-07-25|      12|         CLOSED|
|       9149|2013-07-25|      13|PENDING_PAYMENT|
|       9842|2013-07-25|      14|     PROCESSING|
|       2568|2013-07-25|      15|       COMPLETE|
|       7276|2013-07-25|      16|PENDING_PAYMENT|
|       2667|2013-07-25|      17|       COMPLETE|


In [13]:
#create temp table
ordersDF.registerTempTable("ordersDF_table")

In [14]:
sqlContext.sql("select * from ordersDF_table").show()

+-----------+----------+--------+---------------+
|customer_id|order_date|order_id|   order_status|
+-----------+----------+--------+---------------+
|      11599|2013-07-25|       1|         CLOSED|
|        256|2013-07-25|       2|PENDING_PAYMENT|
|      12111|2013-07-25|       3|       COMPLETE|
|       8827|2013-07-25|       4|         CLOSED|
|      11318|2013-07-25|       5|       COMPLETE|
|       7130|2013-07-25|       6|       COMPLETE|
|       4530|2013-07-25|       7|       COMPLETE|
|       2911|2013-07-25|       8|     PROCESSING|
|       5657|2013-07-25|       9|PENDING_PAYMENT|
|       5648|2013-07-25|      10|PENDING_PAYMENT|
|        918|2013-07-25|      11| PAYMENT_REVIEW|
|       1837|2013-07-25|      12|         CLOSED|
|       9149|2013-07-25|      13|PENDING_PAYMENT|
|       9842|2013-07-25|      14|     PROCESSING|
|       2568|2013-07-25|      15|       COMPLETE|
|       7276|2013-07-25|      16|PENDING_PAYMENT|
|       2667|2013-07-25|      17|       COMPLETE|


#### to set no:of tasks for a job

In [15]:
sqlContext.setConf("spark.sql.shuffle.partitions","2")

* Get daily revenue by product considering completed and closed orders.
	- PRODUCTS have to be read from local file system.DataFrame need to be created
	- Join Orders,ORDER_ITEMS
	- Filter on ORDER_STATUS
* Data need to be sorted by ascending order by date and thendescending order by revenue computed for each product for each day
	- Sort data by order_date in ascending order and then daily revenue per product in descending order

In [16]:
#read product data from local file system and convert to table
productsRaw = open("/home/pi/shared/retail_db/products/part-00000").read().splitlines()
productsRDD = sc.parallelize(productsRaw)
productsRDD.take(10)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat",
 "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 '5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet',
 "6,2,Jordan Men's VI Retro TD Football Cleat,,134.99,http://images.acmesports.sports/Jordan+Men%27s+VI+Retro+TD+Football+Cleat",
 '7,2,Schutt Youth Recruit Hybrid Custom Football H,,99.99,http://images.acmesports.sports/Schutt+Youth+Recruit+Hybrid+Custom+Football+Helmet+20

In [17]:
#convert RDD to DF and then to table
from pyspark.sql import Row
productsDF=productsRDD.map(lambda p:Row(product_id=int(p.split(",")[0]),product_name=p.split(",")[2])).toDF()
productsDF.registerTempTable("products")

In [18]:
#read products
sqlContext.sql("select * from products").show()

+----------+--------------------+
|product_id|        product_name|
+----------+--------------------+
|         1|Quest Q64 10 FT. ...|
|         2|Under Armour Men'...|
|         3|Under Armour Men'...|
|         4|Under Armour Men'...|
|         5|Riddell Youth Rev...|
|         6|Jordan Men's VI R...|
|         7|Schutt Youth Recr...|
|         8|Nike Men's Vapor ...|
|         9|Nike Adult Vapor ...|
|        10|Under Armour Men'...|
|        11|Fitness Gear 300 ...|
|        12|Under Armour Men'...|
|        13|Under Armour Men'...|
|        14|Quik Shade Summit...|
|        15|Under Armour Kids...|
|        16|Riddell Youth 360...|
|        17|Under Armour Men'...|
|        18|Reebok Men's Full...|
|        19|Nike Men's Finger...|
|        20|Under Armour Men'...|
+----------+--------------------+
only showing top 20 rows



In [19]:
#read order
sqlContext.sql("use retail_db_txt")
sqlContext.sql("select * from orders").show()

++
||
++
++

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|         status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:

In [22]:
sqlContext.sql("select * from order_items").show()

+-------------+--------+----------+--------+--------+-------------+
|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+-------------+--------+----------+--------+--------+-------------+
|            1|       1|       957|       1|  299.98|       299.98|
|            2|       2|      1073|       1|  199.99|       199.99|
|            3|       2|       502|       5|   250.0|         50.0|
|            4|       2|       403|       1|  129.99|       129.99|
|            5|       4|       897|       2|   49.98|        24.99|
|            6|       4|       365|       5|  299.95|        59.99|
|            7|       4|       502|       3|   150.0|         50.0|
|            8|       4|      1014|       4|  199.92|        49.98|
|            9|       5|       957|       1|  299.98|       299.98|
|           10|       5|       365|       5|  299.95|        59.99|
|           11|       5|      1014|       2|   99.96|        49.98|
|           12|       5|       957|       1|  29

In [29]:
sqlContext.sql("select to_date(o.order_date) order_date,p.product_name,round(sum(oi.subtotal),2) revenue \
from orders o join order_items oi on (o.order_id=oi.order_id) \
              join products p on (p.product_id=oi.product_id) \
where o.status in ('COMPLETE','CLOSED') \
group by to_date(o.order_date),p.product_name \
order by order_date,revenue desc").show()

+----------+--------------------+-------+
|order_date|        product_name|revenue|
+----------+--------------------+-------+
|2013-07-25|Field & Stream Sp...|5599.72|
|2013-07-25|Nike Men's Free 5...|5099.49|
|2013-07-25|Diamondback Women...| 4499.7|
|2013-07-25|Perfect Fitness P...|3359.44|
|2013-07-25|Pelican Sunstream...|2999.85|
|2013-07-25|O'Brien Men's Neo...|2798.88|
|2013-07-25|Nike Men's CJ Eli...|1949.85|
|2013-07-25|Nike Men's Dri-FI...| 1650.0|
|2013-07-25|Under Armour Girl...|1079.73|
|2013-07-25|Bowflex SelectTec...| 599.99|
|2013-07-25|Elevation Trainin...| 319.96|
|2013-07-25|Titleist Pro V1 H...| 207.96|
|2013-07-25|Nike Men's Kobe I...| 199.99|
|2013-07-25|Cleveland Golf Wo...| 119.99|
|2013-07-25|TYR Boys' Team Di...| 119.97|
|2013-07-25|Merrell Men's All...| 109.99|
|2013-07-25|LIJA Women's Butt...|  108.0|
|2013-07-25|Nike Women's Lege...|  100.0|
|2013-07-25|Team Golf Tenness...|  99.96|
|2013-07-25|Bridgestone e6 St...|  95.97|
+----------+--------------------+-

In [30]:
sqlContext.sql("show tables").show()

+-------------+--------------+-----------+
|     database|     tableName|isTemporary|
+-------------+--------------+-----------+
|retail_db_txt|     customers|      false|
|retail_db_txt|   order_items|      false|
|retail_db_txt|        orders|      false|
|             |ordersdf_table|       true|
|             |      products|       true|
+-------------+--------------+-----------+



#### Write Spark Application - Saving Data Frame to Hive tables

* Use Hive and store the output to hive database
	- Get order_date,product_name,daily_revenue_per_product and save into Hive table using ORC file format

In [39]:
sqlContext.sql("create database ameen_daily_revenue")

DataFrame[]

In [44]:
sqlContext.sql("create table daily_revenue (order_date string,product_name string,revenue float) stored as orc")

DataFrame[]

In [45]:
sqlContext.sql("select * from ameen_daily_revenue.daily_revenue")

DataFrame[order_date: string, product_name: string, revenue: float]

In [55]:
dailyRevenue_df=sqlContext.sql("select to_date(o.order_date) order_date,p.product_name,round(sum(oi.subtotal),2) revenue \
from orders o join order_items oi on (o.order_id=oi.order_id) \
              join products p on (p.product_id=oi.product_id) \
where o.status in ('COMPLETE','CLOSED') \
group by to_date(o.order_date),p.product_name \
order by order_date,revenue desc")

In [66]:
dailyRevenue_df.write.insertInto("ameen_daily_revenue.daily_revenue")

In [75]:
sqlContext.sql("select * from ameen_daily_revenue.daily_revenue").show()

+----------+--------------------+-------+
|order_date|        product_name|revenue|
+----------+--------------------+-------+
|2014-01-21|Mio ALPHA Heart R...|  199.0|
|2014-01-21|Nike Men's Comfor...| 179.96|
|2014-01-21|Titleist Pro V1x ...| 155.97|
|2014-01-21|Glove It Women's ...| 153.93|
|2014-01-21|Nike Men's Deutsc...|  150.0|
|2014-01-21|Merrell Women's G...| 129.99|
|2014-01-21|LIJA Women's Mid-...|  100.0|
|2014-01-21|Team Golf Pittsbu...|  99.96|
|2014-01-21|Titleist Pro V1x ...|  95.98|
|2014-01-21|Bridgestone e6 St...|  95.97|
|2014-01-21|Glove It Imperial...|  63.96|
|2014-01-21|Nike Women's Temp...|   60.0|
|2014-01-21|Bag Boy Beverage ...|  49.98|
|2014-01-21|Team Golf New Eng...|  49.98|
|2014-01-21|Glove It Urban Br...|  47.97|
|2014-01-21|Bridgestone e6 St...|  31.99|
|2014-01-21|Nike Dri-FIT Crew...|   22.0|
|2014-01-22|Field & Stream Sp...|11999.4|
|2014-01-22|Diamondback Women...|6599.56|
|2014-01-22|Nike Men's Dri-FI...| 6000.0|
+----------+--------------------+-

In [82]:
#method2
sqlContext.sql("create table ameen_daily_revenue.daily_revenue_2 (order_date string,product_name string,revenue float) stored as orc")

DataFrame[]

In [91]:
sqlContext.sql("insert into ameen_daily_revenue.daily_revenue_2 select to_date(o.order_date) order_date,p.product_name,round(sum(oi.subtotal),2) revenue \
from orders o join order_items oi on (o.order_id=oi.order_id) \
              join products p on (p.product_id=oi.product_id) \
where o.status in ('COMPLETE','CLOSED') \
group by to_date(o.order_date),p.product_name \
order by order_date,revenue desc")

DataFrame[]

In [93]:
sqlContext.sql("select * from ameen_daily_revenue.daily_revenue_2").show()

+----------+--------------------+-------+
|order_date|        product_name|revenue|
+----------+--------------------+-------+
|2013-07-25|Field & Stream Sp...|5599.72|
|2013-07-25|Nike Men's Free 5...|5099.49|
|2013-07-25|Diamondback Women...| 4499.7|
|2013-07-25|Perfect Fitness P...|3359.44|
|2013-07-25|Pelican Sunstream...|2999.85|
|2013-07-25|O'Brien Men's Neo...|2798.88|
|2013-07-25|Nike Men's CJ Eli...|1949.85|
|2013-07-25|Nike Men's Dri-FI...| 1650.0|
|2013-07-25|Under Armour Girl...|1079.73|
|2013-07-25|Bowflex SelectTec...| 599.99|
|2013-07-25|Elevation Trainin...| 319.96|
|2013-07-25|Titleist Pro V1 H...| 207.96|
|2013-07-25|Nike Men's Kobe I...| 199.99|
|2013-07-25|Cleveland Golf Wo...| 119.99|
|2013-07-25|TYR Boys' Team Di...| 119.97|
|2013-07-25|Merrell Men's All...| 109.99|
|2013-07-25|LIJA Women's Butt...|  108.0|
|2013-07-25|Nike Women's Lege...|  100.0|
|2013-07-25|Team Golf Tenness...|  99.96|
|2013-07-25|Bridgestone e6 St...|  95.97|
+----------+--------------------+-

### Data Frame Operations

    - show() --preview data
    - write.insertInto() -- to write to hive table
    - select
    - filter
    - join
    

In [95]:
dailyRevenue_df.show(10)

+----------+--------------------+-------+
|order_date|        product_name|revenue|
+----------+--------------------+-------+
|2013-07-25|Field & Stream Sp...|5599.72|
|2013-07-25|Nike Men's Free 5...|5099.49|
|2013-07-25|Diamondback Women...| 4499.7|
|2013-07-25|Perfect Fitness P...|3359.44|
|2013-07-25|Pelican Sunstream...|2999.85|
|2013-07-25|O'Brien Men's Neo...|2798.88|
|2013-07-25|Nike Men's CJ Eli...|1949.85|
|2013-07-25|Nike Men's Dri-FI...| 1650.0|
|2013-07-25|Under Armour Girl...|1079.73|
|2013-07-25|Bowflex SelectTec...| 599.99|
+----------+--------------------+-------+
only showing top 10 rows



In [96]:
dailyRevenue_df.printSchema()

root
 |-- order_date: date (nullable = true)
 |-- product_name: string (nullable = true)
 |-- revenue: double (nullable = true)



In [109]:
dailyRevenue_df.write.saveAsTable("ameen_daily_revenue.daily_revenue_3")

In [111]:
sqlContext.sql("select * from ameen_daily_revenue.daily_revenue_3").show()

+----------+--------------------+-------+
|order_date|        product_name|revenue|
+----------+--------------------+-------+
|2013-07-25|Field & Stream Sp...|5599.72|
|2013-07-25|Nike Men's Free 5...|5099.49|
|2013-07-25|Diamondback Women...| 4499.7|
|2013-07-25|Perfect Fitness P...|3359.44|
|2013-07-25|Pelican Sunstream...|2999.85|
|2013-07-25|O'Brien Men's Neo...|2798.88|
|2013-07-25|Nike Men's CJ Eli...|1949.85|
|2013-07-25|Nike Men's Dri-FI...| 1650.0|
|2013-07-25|Under Armour Girl...|1079.73|
|2013-07-25|Bowflex SelectTec...| 599.99|
|2013-07-25|Elevation Trainin...| 319.96|
|2013-07-25|Titleist Pro V1 H...| 207.96|
|2013-07-25|Nike Men's Kobe I...| 199.99|
|2013-07-25|Cleveland Golf Wo...| 119.99|
|2013-07-25|TYR Boys' Team Di...| 119.97|
|2013-07-25|Merrell Men's All...| 109.99|
|2013-07-25|LIJA Women's Butt...|  108.0|
|2013-07-25|Nike Women's Lege...|  100.0|
|2013-07-25|Team Golf Tenness...|  99.96|
|2013-07-25|Bridgestone e6 St...|  95.97|
+----------+--------------------+-

In [117]:
dailyRevenue_df.select('order_date','revenue').show()

+----------+-------+
|order_date|revenue|
+----------+-------+
|2013-07-25|5599.72|
|2013-07-25|5099.49|
|2013-07-25| 4499.7|
|2013-07-25|3359.44|
|2013-07-25|2999.85|
|2013-07-25|2798.88|
|2013-07-25|1949.85|
|2013-07-25| 1650.0|
|2013-07-25|1079.73|
|2013-07-25| 599.99|
|2013-07-25| 319.96|
|2013-07-25| 207.96|
|2013-07-25| 199.99|
|2013-07-25| 119.99|
|2013-07-25| 119.97|
|2013-07-25| 109.99|
|2013-07-25|  108.0|
|2013-07-25|  100.0|
|2013-07-25|  99.96|
|2013-07-25|  95.97|
+----------+-------+
only showing top 20 rows



In [125]:
dailyRevenue_df.filter(dailyRevenue_df['order_date']=='2013-07-25').show()

+----------+--------------------+-------+
|order_date|        product_name|revenue|
+----------+--------------------+-------+
|2013-07-25|Field & Stream Sp...|5599.72|
|2013-07-25|Nike Men's Free 5...|5099.49|
|2013-07-25|Diamondback Women...| 4499.7|
|2013-07-25|Perfect Fitness P...|3359.44|
|2013-07-25|Pelican Sunstream...|2999.85|
|2013-07-25|O'Brien Men's Neo...|2798.88|
|2013-07-25|Nike Men's CJ Eli...|1949.85|
|2013-07-25|Nike Men's Dri-FI...| 1650.0|
|2013-07-25|Under Armour Girl...|1079.73|
|2013-07-25|Bowflex SelectTec...| 599.99|
|2013-07-25|Elevation Trainin...| 319.96|
|2013-07-25|Titleist Pro V1 H...| 207.96|
|2013-07-25|Nike Men's Kobe I...| 199.99|
|2013-07-25|Cleveland Golf Wo...| 119.99|
|2013-07-25|TYR Boys' Team Di...| 119.97|
|2013-07-25|Merrell Men's All...| 109.99|
|2013-07-25|LIJA Women's Butt...|  108.0|
|2013-07-25|Nike Women's Lege...|  100.0|
|2013-07-25|Team Golf Tenness...|  99.96|
|2013-07-25|Bridgestone e6 St...|  95.97|
+----------+--------------------+-

In [ ]:
#df.write.json()
#df.write.save(fn,type)

In [126]:
help(dailyRevenue_df)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(builtins.object)
 |  DataFrame(jdf, sql_ctx)
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`::
 |  
 |      people = spark.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the :class:`DataFrame`, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SparkSession
 |      people = spark.read.parquet("...")
 |      department = spark.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId == department.id) \
 |        .groupBy(department.name, "gender").agg({"salary": "av

In [133]:
dailyRevenue_df.toJSON().take(10)

['{"order_date":"2013-07-25","product_name":"Field & Stream Sportsman 16 Gun Fire Safe","revenue":5599.72}',
 '{"order_date":"2013-07-25","product_name":"Nike Men\'s Free 5.0+ Running Shoe","revenue":5099.49}',
 '{"order_date":"2013-07-25","product_name":"Diamondback Women\'s Serene Classic Comfort Bi","revenue":4499.7}',
 '{"order_date":"2013-07-25","product_name":"Perfect Fitness Perfect Rip Deck","revenue":3359.44}',
 '{"order_date":"2013-07-25","product_name":"Pelican Sunstream 100 Kayak","revenue":2999.85}',
 '{"order_date":"2013-07-25","product_name":"O\'Brien Men\'s Neoprene Life Vest","revenue":2798.88}',
 '{"order_date":"2013-07-25","product_name":"Nike Men\'s CJ Elite 2 TD Football Cleat","revenue":1949.85}',
 '{"order_date":"2013-07-25","product_name":"Nike Men\'s Dri-FIT Victory Golf Polo","revenue":1650.0}',
 '{"order_date":"2013-07-25","product_name":"Under Armour Girls\' Toddler Spine Surge Runni","revenue":1079.73}',
 '{"order_date":"2013-07-25","product_name":"Bowflex 